## Project 1.3 - Test Functions 

Group: Emmy Noether

Students: Janik Rausch (628334), Camilo Tello Breuer (633291), Ida Wöstheinreich (628428)

In [1]:
using LinearAlgebra,Random,Printf,FFTW,JLD,Plots,LaTeXStrings,Colors
include("Ground_state.jl"); include("Evolution.jl")

simulate_2D

### Two important properties
- **The unitarity of integrators**\
    The exact evolution operator is a unitary operator, i.e. it satisfies 
    $$U_\text{ex}^\dagger U_\text{ex} =1.$$
    The unitarity property in the context of quantum mechanics is also often called $\textit{conservation of total probability}$, because this property is equivalent to the statement that the norm of states is conserved by the time evolution. 

- **The energy conservation of integrators**\
    The exact evolution operator satisfies
    $$U_\text{ex}^\dagger \hat{H}U_\text{ex} =\hat{H}$$
    This property implies that the expectation value of the energy does not depend on time, i.e. it is conserved.

In [55]:
function test_unitarity(N::Int, μ::Number, ϵ::Number, τ::Number, σ::Number , method::String; tol_cg::Number=1e-10, maxiters_cg::Int=10000)
    m, k = [(1/ϵ,),(1/ϵ,0),(1/ϵ,0, 0)], [(1,),(1,0),(1,0,0)] # length(m) == ndims(n) 
    function test_engine(shape::Tuple{Vararg{Int}}) 
        print(m[length(shape)], k[length(shape)])  
        n = lattice(N, length(shape))
        V = potential(μ, ϵ, n)
        ψ_0 = wave_packet(n, m[length(shape)], σ, k[length(shape)])
        integrator = choose_integrator(method; tol_cg, maxiters_cg)
        norm_ψ_0 = norm(ψ_0)
        ψ = integrator(μ, ϵ, ψ_0, V, τ)
        norm_ψ = norm(ψ)
        residue = abs(norm_ψ - norm_ψ_0) 
        @printf "residue = %.3e" residue 
    end
    print("\nConservation of the Norm of the states with the "); print(method); print(" method after a time step(τ = "); print(τ) 
    print("): \n");
    print("D=1: "); test_engine((N,))
    print("\nD=2: "); test_engine((N,N))
    #print("\nD=3: "); test_engine((N,N,N)); print("\n"))   
end 
function test_energy_conservation(N::Int, μ::Number, ϵ::Number, τ::Number, σ::Number, method::String; tol_cg::Number=1e-10, maxiters_cg::Int=10000)
    m, k = [(1/ϵ,),(1/ϵ,0),(1/ϵ,0, 0)], [(1,),(1,0),(1,0,0)]
    function test_engine(shape::Tuple{Vararg{Int}}) 
        print(m[length(shape)], k[length(shape)])  
        n = lattice(N, length(shape))
        V = potential(μ, ϵ, n)
        ψ_0 = wave_packet(n, m[length(shape)], σ, k[length(shape)])
        integrator = choose_integrator(method; tol_cg, maxiters_cg)
        E_0,_ = energy_expectation(ψ_0, μ, ϵ)
        ψ = integrator(μ, ϵ, ψ_0, V, τ)
        E,_= energy_expectation(ψ, μ, ϵ)
        residue = abs(E - E_0)
        @printf "residue = %.3e" residue
    end
    print("\nConservation of the energy expectation with the "); print(method); print(" method after a time step(τ = "); print(τ)
    print("): \n"); 
    print("D=1: "); test_engine((N,))
    print("\nD=2: "); test_engine((N,N))
    #print("\nD=3: "); test_engine((N,N,N)); print("\n"))   
end

test_energy_conservation (generic function with 3 methods)

Not all approximated integrators preserve unitarity and energy conservation exactly.
For example, the `Euler integrator` is neither unitary nor energy-preserving. The residues are slightly higher for higher dimensions.  
But both properties are recovered in the τ → 0 limit:

In [5]:
N = 200
τ = [0.5, 0.05, 0.0005]
μ, ϵ = 20, sqrt(0.001)
σ = 10
method = ["euler", "crank-nicolson", "strang-splitting"]
for t in τ
    test_unitarity(N, μ, ϵ, t, σ, method[1])
    test_energy_conservation(N, μ, ϵ, t, σ, method[1])
end


Conservation of the Norm of the states with the euler method after a time step(τ = 0.5): 
D=1: 

residue = 1.093e+01
D=2: 

residue = 1.101e+01
Conservation of the energy expectation with the euler method after a time step(τ = 0.5): 
D=1: residue = 3.440e+03
D=2: residue = 3.509e+03


Conservation of the Norm of the states with the euler method after a time step(τ = 0.05): 
D=1: residue = 5.532e-01
D=2: residue = 5.593e-01
Conservation of the energy expectation with the euler method after a time step(τ = 0.05): 
D=1: residue = 3.440e+01
D=2: residue = 3.509e+01
Conservation of the Norm of the states with the euler method after a time step(τ = 0.0005): 
D=1: residue = 7.063e-05
D=2: residue = 7.157e-05
Conservation of the energy expectation with the euler method after a time step(τ = 0.0005): 
D=1: residue = 3.440e-03
D=2: residue = 3.509e-03

But the `Crank-Nicolson` integrator is unitary and conserves the energy. 
The residues for these method are consistently of order $10^{-10}$ or smaller for both tests.
The size of the residues does not depend on the size of τ:

In [6]:
N = 200
τ = [0.5, 0.05, 0.0005]
μ, ϵ = 20, sqrt(0.001)
σ = 10
method = ["euler", "crank-nicolson", "strang-splitting"]
for t in τ
    test_unitarity(N, μ, ϵ, t, σ, method[2])
    test_energy_conservation(N, μ, ϵ, t, σ, method[2])
end


Conservation of the Norm of the states with the crank-nicolson method after a time step(τ = 0.5): 
D=1: 

residue = 4.610e-12
D=2: 

residue = 3.377e-13
Conservation of the energy expectation with the crank-nicolson method after a time step(τ = 0.5): 
D=1: residue = 5.906e-11


D=2: residue = 3.158e-12


Conservation of the Norm of the states with the crank-nicolson method after a time step(τ = 0.05): 
D=1: residue = 3.239e-12
D=2: 

residue = 4.456e-13
Conservation of the energy expectation with the crank-nicolson method after a time step(τ = 0.05): 
D=1: residue = 2.859e-11
D=2: residue = 1.489e-11


Conservation of the Norm of the states with the crank-nicolson method after a time step(τ = 0.0005): 
D=1: residue = 1.354e-14
D=2: 

residue = 3.275e-14
Conservation of the energy expectation with the crank-nicolson method after a time step(τ = 0.0005): 
D=1: residue = 4.146e-12
D=2: residue = 9.805e-13

The `Strang-splitting` integrator is unitary and but does not conserve the energy. 
The residue for the unitary test are consistently of order $10^{-11}$ or smaller and
their size does not depend on the size of τ. Energy conservation is recovered in the τ → 0 limit:

In [46]:
function test_unitarity_ss(N::Int, n::Array, μ::Number, ϵ::Number, τ::Number, ψ_0::Array; method::String="strang-splitting", tol_cg::Number=1e-10, maxiters_cg::Int=10000)
    integrator = choose_integrator(method; tol_cg, maxiters_cg)
    V = potential(μ, ϵ, n)
    norm_ψ_0 = norm(ψ_0)
    ψ = integrator(μ, ϵ, ψ_0, V, τ)
    norm_ψ = norm(ψ)
    residue = abs(norm_ψ - norm_ψ_0)
    print("\nThe difference between the norm of states before and after a time step(τ = "); print(τ) 
    print("):\nresidue = "); print(residue)
end 
function test_energy_conservation_ss(N::Int, n::Array, μ::Number, ϵ::Number, τ::Number, ψ_0::Array; method::String="strang-splitting", tol_cg::Number=1e-10, maxiters_cg::Int=10000)
    integrator = choose_integrator(method; tol_cg, maxiters_cg)
    V = potential(μ, ϵ, n)
    E_0,_ = energy_expectation(ψ_0, μ, ϵ)
    ψ = integrator(μ, ϵ, ψ_0, V, τ)
    E,_= energy_expectation(ψ, μ, ϵ)
    residue = abs(E - E_0)
    print("\nThe difference between the energy expectation before and after a time step(τ = "); print(τ)
    print("):\nresidue = "); print(residue); print("\n")
end

test_energy_conservation_ss (generic function with 3 methods)

In [48]:
N = 200
τ = [0.5, 0.05, 0.0005]
μ, ϵ = 20, sqrt(0.001)
m, k = [(1/ϵ,),(1/ϵ,0), (1/ϵ,0,0)], [(1,),(1,0), (1/ϵ,0,1)]
σ = 10
for i in 1:2
    print("\nD = "); print(i); print("\n")
    n = lattice(N, i)
    ψ_0 = wave_packet(lattice(N, i), m[i], σ, k[i])
    for t in τ
    test_unitarity_ss(N, n, μ, ϵ, t, ψ_0)
    test_energy_conservation_ss(N, n, μ, ϵ, t, ψ_0)
    end
end    



D = 1

The difference between the norm of states before and after a time step(τ = 0.5):
residue = 2.220446049250313e-16
The difference between the energy expectation before and after a time step(τ = 0.5):
residue = 0.6705691151501227

The difference between the norm of states before and after a time step(τ = 0.05):
residue = 0.0
The difference between the energy expectation before and after a time step(τ = 0.05):
residue = 0.0013618594457902589

The difference between the norm of states before and after a time step(τ = 0.0005):
residue = 2.220446049250313e-16
The difference between the energy expectation before and after a time step(τ = 0.0005):
residue = 1.354560907884661e-9

D = 2

The difference between the norm of states before and after a time step(τ = 

0.5):
residue = 0.0
The difference between the energy expectation before and after a time step(τ = 

0.5):
residue = 0.5884506296855037



The difference between the norm of states before and after a time step(τ = 0.05):
residue = 0.0


The difference between the energy expectation before and after a time step(τ = 0.05):
residue = 0.0013455511857856095

The difference between the norm of states before and after a time step(τ = 

...

0.5):
residue = 0.5815978930321783



The difference between the norm of states before and after a time step(τ = 0.05):
residue = 0.0


The difference between the energy expectation before and after a time step(τ = 0.05):
residue = 0.0003615687832514425

The difference between the norm of states before and after a time step(τ = 

0.0005):
residue = 0.0
The difference between the energy expectation before and after a time step(τ = 

0.0005):
residue = 3.2085978318718844e-10


For the `Strang-splitting` and the `Crank-Nicolson` integrator the variation of the energy expectation and the norm of states is lower for higher dimensions.